In [ ]:
# Q1
{
  "name" : "56f843790c93",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "jeW-UWoYR8KBg5-anwWq-Q",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}

In [ ]:
"build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"

In [104]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [105]:
# Q2
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [106]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [107]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:22<00:00, 42.39it/s]


In [108]:
# Which function do you use for adding your data to elastic?

## We use index function

In [109]:
from openai import OpenAI
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [110]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question", "text"],
                        "boost": 4,
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs, response

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [111]:
def rag(query):
    search_results, response = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer, response

In [112]:
query = 'How do I execute a command in a running docker container?'
ans, response = rag(query)

In [113]:
response

ObjectApiResponse({'took': 87, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 86.67784, 'hits': [{'_index': 'course-questions', '_id': 'SRacTJABVfXUHjPwxrd5', '_score': 86.67784, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'aBacTJABVfXUHjPwybc1', '_score': 85.69645, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo co

In [114]:
ans

'To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps:\n\n1. List the running containers to find the `container-id`:\n   ```\n   docker ps\n   ```\n\n2. Execute a command in the specific container using `docker exec`. For example, to start a bash session in the container, use:\n   ```\n   docker exec -it <container-id> bash\n   ```\n\nThis will allow you to interact with the running container and run any required commands within it.'

In [115]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question", "text"],
                        "boost": 4,
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response

In [116]:
def rag(query):
    search_results= elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer, search_results

In [117]:
query = 'How do I debug a docker container?'
ans, search_results = rag(query)

In [118]:
# response.choices[0].message.content
search_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [119]:
## Q5
context_template = """
Q: {question}
A: {text}
""".strip()


In [120]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question", "text"],
                        "boost": 4,
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context = ""
    
    for doc in search_results:
        context = f"""
                                Q: {doc['question']}\n\n
                                A: {doc['text']}\n\n
                            """
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response

In [121]:
def rag(query):
    search_results= elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer, prompt

In [122]:
query = 'How do I execute a command in a running docker container?'
ans, prompt = rag(query)
ans

ChatCompletion(id='chatcmpl-9dnYBRZ6w5P4biTfSfyel32alUahb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To execute a command in a running Docker container, you typically use the `docker exec` command. Unfortunately, the provided CONTEXT did not contain specific information or instructions about executing commands in a running Docker container. \n\nHowever, in general, you can use the following command structure:\n\n```sh\ndocker exec -it <container_id_or_name> <command>\n```\n\nFor example, to run a shell inside a container, you can use:\n\n```sh\ndocker exec -it <container_id_or_name> /bin/bash\n```\n\nOr, if the container has sh instead of bash:\n\n```sh\ndocker exec -it <container_id_or_name> /bin/sh\n```\n\nReplace `<container_id_or_name>` with the actual ID or name of your running Docker container and `<command>` with the command you want to execute.', role='assistant', function_call=None, tool_calls=None))], created=1719272

In [123]:
len(prompt)

915

In [124]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\n\n                                Q: I can’t create the environment on AWS Elastic Beanstalk with the command proposed during the video\n\n\n                                A: I struggled with the command :\neb init -p docker tumor-diagnosis-serving -r eu-west-1\nWhich resulted in an error when running : eb local run --port 9696\nERROR: NotSupportedError - You can use "eb local" only with preconfigured, generic and multicontainer Docker platforms.\nI replaced it with :\neb init -p "Docker running on 64bit Amazon Linux 2" tumor-diagnosis-serving -r eu-west-1\nThis allowed the recognition of the Dockerfile and the build/run of the docker container.\nAdded by Mélanie Fouesnard'

In [125]:
import tiktoken

In [126]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [127]:
encoding.encode(prompt)

[63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 558,
 8470,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 364,
 107036,
 25,
 3253,
 621,
 357,
 15792,
 261,
 6348,
 306,
 261,
 6788,
 62275,
 9282,
 1715,
 10637,
 50738,
 1402,
 2419,
 1486,
 25,
 357,
 665,
 1573,
 2501,
 290,
 5870,
 402,
 40229,
 88516,
 2439,
 27098,
 2082,
 483,
 290,
 6348,
 17994,
 3354,
 290,
 3823,
 2499,
 2419,
 355,
 25,
 357,
 52720,
 483,
 290,
 6348,
 10039,
 1113,
 6327,
 533,
 79,
 62275,
 40770,
 2469,
 34930,
 12635,
 199516,
 533,
 81,
 5658,
 78416,
 12,
 16,
 198,
 34405,
 32871,
 306,
 448,
 2915,
 1261,
 6788,
 712,
 17254,
 2698,
 2461,
 2230,
 447,
 220,
 48806,
 21,
 198,
 7751,
 25,
 4037,
 33833,
 2255,
 533,
 1608,
 665,
 1199,
 392,
 1113,
 2698,
 1,
 1606,
 483,
 876,
 129656,
 11,
 21081,
 326,
 74200,
 645,
 2573,
 91238,
 18921,
 558,
 40,
 21009,
 480,
 483,
 10039,
 1113,
